<a href="https://colab.research.google.com/github/RCSnyder/Sales_Forecasting/blob/Dany/Sales_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from datetime import datetime

In [ ]:
#Importing csv files
drive.mount('/content/gdrive')
fp = '/content/gdrive/My Drive/Sales Data'
train     = pd.read_csv("/content/gdrive/My Drive/Sales Data/sales_train.csv")
test      = pd.read_csv('/content/gdrive/My Drive/Sales Data/test.csv')
items     = pd.read_csv('/content/gdrive/My Drive/Sales Data/items.csv')
item_cats = pd.read_csv('/content/gdrive/My Drive/Sales Data/item_categories.csv')
shops     = pd.read_csv('/content/gdrive/My Drive/Sales Data/shops.csv')

Mounted at /content/gdrive


# New Section

In [ ]:
print(train.head())
print(items.head())

        date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0 2013-02-01               0       59    22154      999.00           1.0
1 2013-03-01               0       25     2552      899.00           1.0
2 2013-05-01               0       25     2552      899.00          -1.0
3 2013-06-01               0       25     2554     1709.05           1.0
4 2013-01-15               0       25     2555     1099.00           1.0
                                           item_name  item_id  item_category_id
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0                40
1  !ABBYY FineReader 12 Professional Edition Full...        1                76
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2                40
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3                40
4        ***КОРОБКА (СТЕКЛО)                       D        4                40


In [ ]:
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
grouped = pd.DataFrame(train.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].sum().reset_index())
print(group.head())
print(grouped.head())
print(group.iloc[0])
print(grouped.iloc[0])

                               item_cnt_day
                                        sum
date_block_num shop_id item_id             
0              0       32               6.0
                       33               3.0
                       35               1.0
                       43               1.0
                       51               2.0
   shop_id  date_block_num  item_cnt_day
0        0               0        5578.0
1        0               1        6127.0
2        1               0        2947.0
3        1               1        3364.0
4        2               0        1146.0
item_cnt_day  sum    6.0
Name: (0, 0, 32), dtype: float64
shop_id              0.0
date_block_num       0.0
item_cnt_day      5578.0
Name: 0, dtype: float64


In [ ]:
# Group total sales by month
train['date'] = pd.to_datetime(train['date'])
monthly_sales_by_item = pd.DataFrame(train.groupby([pd.Grouper(key="date", freq='M'), 'shop_id'])["item_cnt_day"].sum())
#monthly_sales_by_item = monthly_sales_by_item.to_frame()
print(monthly_sales_by_item.describe)


shops['Total Revenue'] = train.groupby("shop_id")["item_price"].sum()
print(shops.head())


<bound method NDFrame.describe of                     item_cnt_day
date       shop_id              
2013-01-31 0              3626.0
           1              1955.0
           2               748.0
           3               670.0
           4              1685.0
...                          ...
2015-12-31 55              982.0
           56              442.0
           57              701.0
           58              559.0
           59              307.0

[1847 rows x 1 columns]>
                        shop_name  shop_id  Total Revenue
0   !Якутск Орджоникидзе, 56 фран        0   5.553869e+06
1   !Якутск ТЦ "Центральный" фран        1   2.926161e+06
2                Адыгея ТЦ "Мега"        2   3.510444e+07
3  Балашиха ТРК "Октябрь-Киномир"        3   2.647262e+07
4        Волжский ТЦ "Волга Молл"        4   3.533539e+07


In [ ]:
# Plot total revenue per shop
fig, ax = plt.subplots()
ax.plot (shops['shop_id'], shops['Total Revenue'])
plt.show()



In [ ]:
#Grouping Shop_Id and Item_Id
from itertools import product
import numpy as np

months = train['date_block_num'].unique()

month_shop_and_item = []
for month in months:
    shops_in_month = train.loc[train['date_block_num']==month, 'shop_id'].unique()
    items_in_month = train.loc[train['date_block_num']==month, 'item_id'].unique()
    month_shop_and_item.append(np.array(list(product(*[shops_in_month, items_in_month, [month]])), dtype='int32'))

In [ ]:
#Create a df and aggregate at the month level
month_shop_and_item_df = pd.DataFrame(np.vstack(month_shop_and_item), columns = ['shop_id', 'item_id', 'date_block_num'], dtype=np.int32)
grouped_by_month = train.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().rename('item_cnt_month').reset_index()


In [ ]:
#Merge to form a new training set
mod_train = pd.merge(month_shop_and_item_df, grouped_by_month, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)

In [ ]:
print(mod_train.head())

   shop_id  item_id  date_block_num  item_cnt_month
0       59    22154               0             1.0
1       59     2552               0             0.0
2       59     2554               0             0.0
3       59     2555               0             0.0
4       59     2564               0             0.0


In [ ]:
mod_train.shape
type(mod_train)
mod_train = mod_train.sample(frac=1)

In [ ]:
#Creating target and feature arrays
X = mod_train.drop('item_cnt_month', axis=1).values
y = mod_train['item_cnt_month'].values
y = y.reshape(-1,1)
#X = X.reshape(-1,1)
X.shape, y.shape

((10913850, 3), (10913850, 1))

In [ ]:
# Encoding categorical data
# used to provide a value to data that can be then used in equations, eg. Friday = 1 etc.
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# the only variable which is not categorical is item_price, hence all other variable will be converted
ct = ColumnTransformer([('encoder', OneHotEncoder(), [0, 1, 2])], remainder = 'passthrough')
X = ct.fit_transform(X)


In [ ]:
#Working with sklearn and Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#Splitting test set and Fitting a model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
reg = LinearRegression()
reg.fit(X_train, y_train)
y_predict = reg.predict(X_test)
reg.score(X_test, y_test)

0.24137872959086526